In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from jumpadjust import JumpAdjust
from noiseadjust import NoiseAdjust
from volatility import VarianceEstimation
from evaluation import Evaluation
from glob import glob 
dfs = []
for f in glob("./mid_1s/??-??_1s.csv.zst"):
    df = pd.read_csv(f, parse_dates=True, index_col=0)
    dfs.append(df)
df = pd.concat(dfs).sort_index()
df = df.loc["2020":"2020-09-02"]

x = df["BTC-USDT_bitfinex"].ffill().bfill().values


In [ ]:
raw_price = df["BTC-USDT_bitfinex"].ffill().bfill().values
results = dict()
for ja in ["truncate_MLE", "truncate", "benchmark"]:
    logprc_ja, _ = JumpAdjust(raw_price).__getattribute__(ja)()
    for na in ["preaveraging", "fracdiff", "benchmark"]:
        logprc_na = NoiseAdjust(logprc_ja).__getattribute__(na)()
        for ev in ["filtration_information"]:
            evaluation = Evaluation(logprc_na).__getattribute__("filtration_information")()
            results[(ja, na, ev)] = evaluation

In [ ]:
pd.Series(results).unstack().sort_values(by="filtration_information")

filtration_information
truncate_MLE preaveraging           -4.449378e-07
benchmark    preaveraging           -4.320090e-07
truncate     preaveraging           -7.329995e-08
             fracdiff               -4.506482e-09
truncate_MLE fracdiff               -1.597965e-09
benchmark    fracdiff               -1.276981e-09
truncate_MLE benchmark              -8.882375e-20
benchmark    benchmark              -8.508607e-20
truncate     benchmark              -8.172954e-20

array([ 1,  1, -1, -1,  5])